# WorldClim data : Historical & future Climatologies (1970-2100) - MAPS
## 4 SSP scenarios and 9 Global Climatic Models (Precipitation, Temperature)
- Data: https://www.worldclim.org/data/index.html
- author: Loïc Duffar https://github.com/loicduffar

This python notebook uses data from the wordclim web site to plot future climatologies maps on a given area. The outputs are stored as image files (png) and raster files (geotif).

### How to PROCEED ?
The data files must be divided into different sub-folders:
- a folder containing the 12 tif files with historical monthly interannual averages (1 file per month of the year)
- a folder with sub-folders for the various future periods (e.g. 1921-1940, 1961-1981, 1981-2100). Each subfolder contains 4 tif files for each model (and each file contains 12 bands for the the different months).
- Optionnaly, the global DEM file can be downloaded from WorldClim to plot the elevation map of the area

Run the first script, then one of the following ones:
- <a href=#section01>1)</a> Customization
- <a href=#section02>2)</a> Plot annual map for a unique scenario/period/model (annual and monthly maps)
- <a href=#section03>3)</a> Generate a bunch of maps of all scenarios/periods/models (annual maps only)
- <a href=#section04>4)</a> Plot the elevation map
- <a href=#section05>5)</a> 

###  <a name=section01></a>1) Customization

NB: The cell below does not produce any result, but its execution is necessary before to run any one of the next cell

Customize the code below before to run it:
- Choose between precipitation or temperature, and between anomaly or not (deviation from historical baseline)
- Set the geographical area to be drawn
- Choose the folders for input data and output files
- Set the file names composition:
  - prefix of the file names
  - List of models, scenarios and time periods
- Set the legend for historical period

- Contour interval for annual map
- vmin, vmax : values to anchor the colormap in order to compare the different maps. If set to (None, None), they are automaticaly computed from the extreme values of each map, in which case the colormaps of the differente maps are not comparable
    - vmin_annual, vmax_annual: minimum and maximum values of the annual data 
    - vmin_monthly, vmax_monthly: minimum and maximum values of the monthly data

- Set the vector files to be drawn for map wrapping (optional): e.g. line/point (river, borders, localities) and/or polygon (watershed or project area). The Coordinate Reference System must be WGS84 (4326).

In [ ]:
"""
Created on March 2021
@author: Loïc Duffar
"""
import numpy as np
import geopandas as gpd
import os
import cartopy

######### Customization
# ----------- data -----------
variable= 'Temperature'# Precipitation or Temperature
plot_anomaly= True      # set it to True to plot anomaly (deviation from baseline) instead of raw data
# ----------- Spatial cropping
# minx, miny, maxx, maxy=[-85.91, 7.81, -82.61, 11.31]# COSTA RICA
# minx, miny, maxx, maxy= [-72.1, 17.3, -68.2, 20.2] # République dominicaine
minx, miny, maxx, maxy= [-74.6, 17.3, -68.2, 20.2] # HISPANIOLA
# ----------- Files -----------
folder_out= r'C:\Users\DUFFAR\Documents\A\ETUDES\2021 Yaque del Sur'
# ........... folder path for the HISTORICAL tif files (WITHOUT "\" CHARACTER AT THE END !!!)
fld_historical= r'X:\1-COMMUN\DIS\Documentation\Hydrologie\Documentation externe\Climat Monde\WorldClim\Historical Climatologies 1970-2000\wc2.1_5m'
# ........... folder path for the FUTURE tif files (WITHOUT "\" CHARACTER AT THE END !!!)
fld_future= r'X:\1-COMMUN\DIS\Documentation\Hydrologie\Documentation externe\Climat Monde\WorldClim\Future data 2021-2100'
# ........... Composition of the tif file names
file_prefix= 'wc2.1_5m_'# e.g. 'wc2.1_5m_'
model_list = ['BCC-CSM2-MR', 'CNRM-CM6-1', 'CNRM-ESM2-1', 'CanESM5', 'GFDL-ESM4', 'IPSL-CM6A-LR', 'MIROC-ES2L', 'MIROC6', 'MRI-ESM2-0']
scenario_list = ['ssp126', 'ssp245', 'ssp370', 'ssp585']
scenario_list_long = ['ssp1-2.6', 'ssp2-4.5', 'ssp3-7.0', 'ssp5-8.5']
period_list = ['2021-2040', '2041-2060', '2061-2080', '2081-2100']
historical_period= '1970-2000'
# ------- MAP -----------
# ........... Min and max values to anchor the MONTHLY colormap, or (None, None) for automatic determination from the data, 
#             in which case the colormaps of the differente maps are not comparable
#    TIPS:
#    - Precipitation monthly: (0, 300).  if (None, None), the values are computed from 2nd and 98th percentiles of the data instead of the extreme values
#    - Temperature monthly: (9, 33)
vmin_monthly, vmax_monthly = (9, 33)
#........ Contour interval for annual map
#    TIPS:
#    - Precipitation Yearly: 300 and 50 or 100 for anomaly
#    - Temperature Yearly: 2 and 0.1 for anomaly
contour_int_annual= 0.1
#........ Min and max values to anchor the ANNUAL colormap, or (None, None) for automatic determination from the data,
#             in which case the colormaps of the differente maps are not comparable
#    TIPS: e.g. (200, 2600) for Precipitation, (11, 31) / (0.6, 3.2) for Temperature / anomaly
#    - Min value should not be greater than the minimum of the data
#    - Max value should be fixed according to contour interval and vmin (even much greater than the maximum of the data)
#    - For Precipitation anomalies, abosolute values of vmin and vmax should be equal (e.g. -800, +800), because the color map is automaticaly set to divergent on either side of zero
vmin_annual, vmax_annual = (0.6, 3.2)
# ........ Vector files (optionnal but very usefull)
#          - Polygon file (shape file) for plotting and average computation (path INCLUDING folder)
polygon_file = r'X:\4-PROJ_INTERNATIONAL\2020_11_25-DO-FAI-GIRE YAQUE DEL SUR\CARTOPLAN\Hidrografia_REPDOM\Subcuencas_YaqueSur.shp'
#          - Shape file (point, line or polygon) for plotting (path INCLUDING FOLDER)
wrapping_file = r'X:\4-PROJ_INTERNATIONAL\2020_11_25-DO-FAI-GIRE YAQUE DEL SUR\CARTOPLAN\Hidrografia_REPDOM\República Dominicana.shp'

######### Initializations
#...... Files
file_prefix = file_prefix + '_' if file_prefix.strip()[-1:]!= '_' else file_prefix.strip()
if variable.upper()== 'PRECIPITATION':
    file_prefix = file_prefix + 'prec'
    unit= 'mm'
elif variable.upper()== 'TEMPERATURE':
    file_prefix = file_prefix + 't'
    unit= '°C'
#...... Map
proj = cartopy.crs.PlateCarree()
nlevels_annual=  int((vmax_annual - vmin_annual) / contour_int_annual) + 1
levels= np.linspace(vmin_annual, vmax_annual, nlevels_annual)
gdf_polyg = gpd.read_file(polygon_file) if os.path.exists(polygon_file) else None
gdf_wrapping = gpd.read_file(wrapping_file) if os.path.exists(wrapping_file) else None
res = '10m'                  # resolution for naturalearth basemap with cartopy: 10m, 50m or 110m (10m = 1/ 10 000 000)

if variable.upper()== 'PRECIPITATION' and plot_anomaly== True:
    cmap = 'seismic_r'
else:
    cmap= 'jet'    # https://matplotlib.org/stable/tutorials/colors/colormaps.html
if variable.upper()== 'PRECIPITATION':
    clabel_fmt= '%d'
else:
    clabel_fmt= '%1.1f'

print('Cartopy version: ', cartopy.__version__)

######### Function definitions ##################################################################
#======= annual map (cartopy library) ============
def plot_annual(da_annual, title='WorldClim - Annual mean map', levels=10, vmin= None, vmax = None, cmap= 'jet', clabel_fmt= '%1.1f',
                shapes= dict(polyg= dict(shape= None, edgecolor='pink', facecolor= 'grey', alpha= 0.4), 
                             vector= dict(shape= None, edgecolor='grey', facecolor='none', color='blue', linewidth=0.5, alpha=1)), 
                folder_out= '', file_out= 'WorldClim - Annual mean map.png',
               ):
    fig = plt.figure(figsize=(18, 14))
    ax = fig.add_subplot(projection= proj)
    #ax.set_extent([lon_min-x_ext, lon_max+x_ext, lat_min-y_ext, lat_max+y_ext], crs=proj)

    #. . . . . . . . plot contours
    cs1 = xr.plot.contourf(da_annual, 'lon', 'lat', levels=levels, vmin= vmin, vmax = vmax,
                               cmap= cmap, alpha =1, cbar_kwargs= dict(shrink=0.6, label= unit, pad=0.2))# contour fills
    cs2 = xr.plot.contour(da_annual, 'lon', 'lat', levels=levels, vmin= vmin, vmax = vmax,
                          linewidths=1)# contour allows to plot contour lines (contourf does not)
    plt.clabel(cs2, fontsize=10, inline=True, fmt= clabel_fmt)

    #. . . . . . . . Map customization (Polygon and shape file defined by user)
    if shapes['polyg']['shape'] is not None:
        shapes['polyg']['shape'].boundary.plot(ax=ax, edgecolor= shapes['polyg']['edgecolor'], facecolor= shapes['polyg']['facecolor'], alpha= shapes['polyg']['alpha'])
    if shapes['vector']['shape'] is not None: 
        shapes['vector']['shape'].plot(ax=ax, edgecolor= shapes['vector']['edgecolor'], facecolor= shapes['vector']['facecolor'], color= shapes['vector']['color'], linewidth= shapes['vector']['linewidth'], alpha= shapes['vector']['alpha']) 

    #. . . . . . . . Map wrapping (NaturalEarth)
    ax.add_feature(cfeature.NaturalEarthFeature('physical', 'land', res, facecolor='antiquewhite', edgecolor='black'),
                    alpha=0.3)
    ax.add_feature(cfeature.NaturalEarthFeature('physical', 'rivers_lake_centerlines', res, facecolor='none', 
                    edgecolor='blue'), alpha=0.5)
    ax.add_feature(cfeature.NaturalEarthFeature('physical', 'rivers_europe', res, facecolor='none', edgecolor='blue'))
    ax.add_feature(cfeature.NaturalEarthFeature('physical', 'lakes', res, facecolor=cfeature.COLORS['water'], edgecolor='blue'), 
                    alpha=0.5)
    ax.add_feature(cfeature.NaturalEarthFeature('physical', 'ocean', res, facecolor= 'aliceblue', edgecolor='black'), 
                    alpha=0.5)
    plt.title(title, loc='center')
    plt.savefig(os.path.join(folder_out, file_out), dpi=150)
    return fig

#======= Maps by month (Facet plot by cartopy library) ============
def plot_by_month(da, title= 'worldclim - Interannual Mean maps by month', 
                  cmap= 'jet', vmin= None, vmax= None,
                  folder_out= '', file_out= 'worldclim - Interannual Mean maps by month.png'):
    grid= xr.plot.imshow(da, x='lon', y='lat', aspect=2, size=2, vmin= vmin, vmax= vmax,# figsize= (15, 18),
    #da.plot(
                         col= 'month', col_wrap= 3, cmap= cmap, robust= True,
                         cbar_kwargs= dict(label= unit, aspect= 30),
                         transform= proj,
                         subplot_kws= dict(projection= proj)# clim(da.min(), da.max())
                  )
    ax= plt.gca()
    #plt.tight_layout()
    # plt.subplots_adjust(wspace=0., hspace=0.2)
    # for im in plt.gca().get_images():
    #     im.set_clim(da.min(), da.max())

    for ax in grid.axes.flat:
    #     ax.add_feature(cfeature.NaturalEarthFeature('physical', 'coastlines', res, facecolor='none', edgecolor='black'))
        ax.add_feature(cfeature.NaturalEarthFeature('physical', 'ocean', res, facecolor= 'aliceblue', edgecolor='black'), alpha=1)
    plt.gcf().suptitle(title, y=1)
    plt.savefig(os.path.join(folder_out, file_out), dpi=150)
    fig= plt.gcf()
    return fig
######### End of functions definition ##################################################################

###  <a name=section02></a>2) Maps for a unique scenario/model/period (annual AND MONTHLY)
This cell allow to plot a unique scenario/model/period, but also provides the monthly maps (wich are not plotted by the next cell)

NB: The cell # 1 of this notebook must be ran before the cell below

Customize the code below before to run it:
- Choose the model, scenario and period, among the lists set in the cell above

In [ ]:
"""
Created on March 2021
@author: Loïc Duffar
"""
import pandas as pd
import hvplot.pandas
import hvplot.xarray
import xarray as xr
import rioxarray
import numpy as np
import matplotlib.pyplot as plt
import cartopy.feature as cfeature
print('Cartopy version: ', cartopy.__version__)

######### Customization 
# ........... Choose the model, scenario and period
model= model_list[0]
scenario= scenario_list[3]
period= period_list[3]

######### Historical Data (mean): Reading and Preprocessing
for i, month in enumerate(range(1, 13)):
    if variable.upper()== 'PRECIPITATION':
        file = os.path.join(fld_historical, file_prefix + '_' + "{:0>2d}".format(month) + '.tif')
    elif variable.upper()== 'TEMPERATURE':
        file = os.path.join(fld_historical, file_prefix + 'avg_' + "{:0>2d}".format(month) + '.tif')
    if i==0:
        da= rioxarray.open_rasterio(os.path.join(fld_historical, file), masked= True)
        da= da.rio.clip_box(minx, miny, maxx, maxy)
    else:
        new_da= rioxarray.open_rasterio(os.path.join(fld_historical, file), masked= True)
        new_da= new_da.rio.clip_box(minx, miny, maxx, maxy)
        liste= list(da['band'].values)
        liste.append(month)
        da= xr.concat([da, new_da], dim= pd.Index( liste, name= 'band'))
# +++++++ Rename x/y coordinates and set spatial dimensions to the new names
da_historical= da.rename(dict(band= 'month', x= 'lon', y= 'lat'))
da_historical.rio.set_spatial_dims(x_dim= 'lon', y_dim= 'lat', inplace= True)
# +++++++ Annual aggregation and Save climatology maps into a geotif file
if variable.upper()== 'PRECIPITATION':
    da_historical_annual = da_historical.sum('month', skipna= False)
elif variable.upper()== 'TEMPERATURE':
    da_historical_annual= da_historical.mean('month', skipna= False)
da_historical_annual.rio.set_spatial_dims(x_dim= 'lon', y_dim= 'lat', inplace= True)

# +++++++ Save maps to a geotif file
file = 'WorldClim - Monthly ' + variable + ' Climatology Map - Historical.tif'
da_historical.rio.to_raster(os.path.join(folder_out, file))
file = 'WorldClim - Annual ' + variable + ' Climatology Map - Historical.tif'
da_historical_annual.rio.to_raster(os.path.join(folder_out, file))

#########  Future Data: Reading and Preprocessing
# +++++++ Read file(s)
path = fld_future + '/' + period
if variable.upper()== 'PRECIPITATION':
    file = os.path.join(path, '_'.join([file_prefix, model, scenario, period]) + '.tif')
    da= rioxarray.open_rasterio(os.path.join(fld_future, file), masked= True)
elif variable.upper()== 'TEMPERATURE':
    file = os.path.join(path, '_'.join([file_prefix + 'min', model, scenario, period]) + '.tif')
    file2 = os.path.join(path, '_'.join([file_prefix + 'max', model, scenario, period]) + '.tif')
    da= rioxarray.open_rasterio(os.path.join(fld_future, file), masked= True)
    da2= rioxarray.open_rasterio(os.path.join(fld_future, file2), masked= True)
# +++++++ Crop raster
da= da.rio.clip_box(minx, miny, maxx, maxy)
# +++++++ Average min/max temperatures
if variable.upper()== 'TEMPERATURE':
    da2= da2.rio.clip_box(minx, miny, maxx, maxy)
    da= (da + da2) / 2
# +++++++ Rename x/y coordinates and set spatial dimensions to the new names
da= da.rename(dict(band= 'month', x='lon', y= 'lat'))
da.rio.set_spatial_dims(x_dim= 'lon', y_dim= 'lat', inplace= True)
# +++++++ Annual aggregation
if variable.upper()== 'PRECIPITATION':
    da_future_annual = da.sum('month', skipna= False)
elif variable.upper()== 'TEMPERATURE':
    da_future_annual = da.mean('month', skipna= False)
#display(da_annual)

if plot_anomaly==True:
    da_to_plot= da_future_annual-da_historical_annual
    base= variable + ' Anomaly '
else:
    da_to_plot= da_future_annual
    base= variable + ' '
#########  Historical Data: Plot annual map (cartopy)
title_annual= 'WorldClim - Annual ' + base
plt.close()
if plot_anomaly== False:
    file_out= title_annual + 'Climatology Map - Historical.png'
    title= title_annual + 'Climatology (' + historical_period + ')'
    fig= plot_annual(da_historical_annual, title= title,  levels=levels, cmap= cmap, vmin= vmin_annual, vmax = vmax_annual,
                    shapes= dict(polyg= dict(shape= gdf_polyg, edgecolor='magenta', facecolor= 'none', alpha= 1), 
                                 vector= dict(shape= gdf_wrapping, edgecolor='grey', facecolor='none', color='none', linewidth=0.5, alpha=1)),
                     folder_out= folder_out, file_out= file_out, clabel_fmt= clabel_fmt )
    plt.show()

#########  Future Data: Plot annual map (cartopy)
plt.close()

file_out=  title_annual +  'Climatology Map - 1 scenario_model_period.png'
title= title_annual + '- ' + '_'.join([period, scenario, model])
fig= plot_annual(da_to_plot, title= title, levels=levels, cmap= cmap, vmin= vmin_annual, vmax = vmax_annual, 
                shapes= dict(polyg= dict(shape= gdf_polyg, edgecolor='magenta', facecolor= 'none', alpha= 1), 
                             vector= dict(shape= gdf_wrapping, edgecolor='grey', facecolor='none', color='none', linewidth=0.5, alpha=1)),
                 folder_out= folder_out, file_out= file_out, clabel_fmt= clabel_fmt)
plt.show()

#########  Historical Data: Monthly Plot (cartopy facet plot)
title_monthly= 'WorldClim - Monthly ' + base
plt.close()
file_out= title_monthly + 'Climatology Map - Historical.png'
title= title_monthly + 'Climatology (' + historical_period + ')'
fig= plot_by_month(da_historical, title= title, cmap= cmap, vmin=vmin_monthly, vmax=vmax_monthly, 
                   folder_out= folder_out, file_out= file_out)
plt.show()

#########  Future Data: Montly Plot (cartopy facet plot)
plt.close()
file_out= title_monthly + 'Climatology Map - Future.png'
title= title_monthly + 'Climatology ' + '_'.join([period, scenario, model])
fig= plot_by_month(da, title= title, cmap= cmap, vmin=vmin_monthly, vmax=vmax_monthly, 
                   folder_out= folder_out, file_out= file_out)
plt.show()

#########  Plot months animation (hvplot)
hv2= da.hvplot(x='lon', y='lat', width= 700, geo= True,  crs= proj,
                coastline= res, 
                tiles= 'CartoLight', # CartoLight, StamenTerrain, StamenTerrainRetina, EsriTerrain, EsriNatGeo
                cmap= cmap, clabel= unit, alpha=0.6,
#                title='WorldClim',
                clim=(da.min(), da.max()),
                groupby= "month",  # Coordinates to group by, for animation plot
                widget_type= "scrubber",
                widget_location= "bottom",
                 )
hv1= da_historical.hvplot(x='lon', y='lat', width= 700, geo= True,  crs= proj,
                coastline= res, 
                tiles= 'CartoLight', # CartoLight, StamenTerrain, StamenTerrainRetina, EsriTerrain, EsriNatGeo
                cmap= cmap, clabel= unit, alpha=0.6,
#                title='WorldClim',
                clim=(da.min(), da.max()),
                groupby= "month",  # Coordinates to group by, for animation plot
                widget_type= "scrubber",
                widget_location= "bottom",
                 )
hv1 + hv2

###  <a name=section03></a>3) Generate a bunch of maps for all scenarios/periods/models (into png files)
NB: The cell # 1 of this notebook must be ran before the cell below

Run this cell and wait the message "Process completed" (one minute or more). 

Maps files are not displayed but stored in the user defined output folder

In [ ]:
"""
Created on March 2021
@author: Loïc Duffar
"""
import warnings
warnings.filterwarnings("ignore")# prevents multiples warnings caused by non updated cartopy module (v 0.17)

import datetime
import pandas as pd
import hvplot.pandas
import hvplot.xarray
import xarray as xr
import rioxarray
import numpy as np
import matplotlib.pyplot as plt
import cartopy.feature as cfeature

######### Initializations
start= datetime.datetime.now()# Start Timer
print('Process in progress (please wait the ending message)')

######### Historical data
# +++++++ Read tif files, crop the rasters (12 months) and store them in a single DataArray
for i, month in enumerate(range(1, 13)):
    if variable.upper()== 'PRECIPITATION':
        file = os.path.join(fld_historical, file_prefix + '_' + "{:0>2d}".format(month) + '.tif')
    elif variable.upper()== 'TEMPERATURE':
        file = os.path.join(fld_historical, file_prefix + 'avg_' + "{:0>2d}".format(month) + '.tif')
    
    if i==0:
        da= rioxarray.open_rasterio(os.path.join(fld_historical, file), masked= True)
        da= da.rio.clip_box(minx, miny, maxx, maxy)
    else:
        new_da= rioxarray.open_rasterio(os.path.join(fld_historical, file), masked= True)
        new_da= new_da.rio.clip_box(minx, miny, maxx, maxy)
        liste= list(da['band'].values)
        liste.append(month)
        da= xr.concat([da, new_da], dim= pd.Index( liste, name= 'band'))
# +++++++ Rename coordinates and set spatial dimensions to 'lon' and 'lat'
da_historical= da.rename(dict(band= 'month', x= 'lon', y= 'lat'))
da_historical.rio.set_spatial_dims(x_dim= 'lon', y_dim= 'lat', inplace= True)
# +++++++ Use da_historical to initialize a new DataArray for the future data, with an additional dimension 'period'
# da_tot= da_historical.expand_dims( {'period': 1, 'model': 1, 'scenario': 1}, 
#                                   axis= range(3, 6))
# da_tot= da_tot.assign_coords( {'period': ('period', [historical_period]), 
#                                'scenario': ('scenario', ['-']),
#                                'model': ('model', ['-']),
#                               })
# display(da_tot)
# +++++++ Annual aggregation
if variable.upper()== 'PRECIPITATION':
    da_historical_annual = da_historical.sum('month', skipna= False)
elif variable.upper()== 'TEMPERATURE':
    da_historical_annual = da_historical.mean('month', skipna= False)
da_historical_annual.rio.set_spatial_dims(x_dim= 'lon', y_dim= 'lat', inplace= True)

# +++++++ Save maps to a geotif file
file = 'WorldClim - Monthly ' + variable + ' Climatology Map - Historical.tif'
da_historical.rio.to_raster(os.path.join(folder_out, file))
file = 'WorldClim - Annual ' + variable + ' Climatology Map - Historical.tif'
da_historical_annual.rio.to_raster(os.path.join(folder_out, file))

#########  Historical Data: Plot annual map (cartopy)
plt.close()
if plot_anomaly== False:
    title= 'WordlClim - ' + variable + ' - ' + 'Annual mean (' + historical_period + ')'
    file_out= 'WorldClim - Annual mean map - Historical.png'
    fig= plot_annual(da_historical_annual, title= title,  levels=levels, cmap= cmap, vmin= vmin_annual, vmax = vmax_annual,
                    shapes= dict(polyg= dict(shape= gdf_polyg, edgecolor='magenta', facecolor= 'none', alpha= 1), 
                                 vector= dict(shape= gdf_wrapping, edgecolor='grey', facecolor='none', color='none', linewidth=0.5, alpha=1)),
                     folder_out= folder_out, file_out= file_out, clabel_fmt= clabel_fmt )

#########  Historical Data: Plot monthly animation (hvplot)
hv= da_historical.hvplot(x='lon', y='lat', width= 700, geo= True,  crs= proj,
                coastline= res, 
                tiles= 'CartoLight', # CartoLight, StamenTerrain, StamenTerrainRetina, EsriTerrain, EsriNatGeo
                cmap= 'jet', clabel= unit, alpha=0.6,
#                title='WorldClim - Historical mean by month',
                clim=(da.min(), da.max()),
                groupby= "month",  # Coordinate to group by, for animation plot
                widget_type= "scrubber",
                widget_location= "bottom",
                 )

######### Future Data: loop for one map per scenario-period-model
plt.ioff()# desactivate interactive plot to prevent displaying maps (maps are saved in png files)
first_array= True
#period_coordinates= [period]
for period in period_list:
    print('Period --------------', period)
    for i, scenario in enumerate(scenario_list):
        for model in model_list:
            path = fld_future + '/' + period
            if variable.upper()== 'PRECIPITATION':
                file = os.path.join(path, '_'.join([file_prefix, model, scenario, period]) + '.tif')
            elif variable.upper()== 'TEMPERATURE':
                file = os.path.join(path, '_'.join([file_prefix + 'min', model, scenario, period]) + '.tif')
                file2 = os.path.join(path, '_'.join([file_prefix + 'max', model, scenario, period]) + '.tif')
            ######### Preprocess
            # +++++++ Read file(s)
            if os.path.exists(os.path.join(fld_future, file)):
                da= rioxarray.open_rasterio(os.path.join(fld_future, file), masked= True)
                if variable.upper()== 'TEMPERATURE':
                    da2= rioxarray.open_rasterio(os.path.join(fld_future, file2), masked= True)
            else:
                continue
                # Skip the plot in case of missing file
            # +++++++ Crop raster
            da= da.rio.clip_box(minx, miny, maxx, maxy)           
            # +++++++ Average min/max temperatures
            if variable.upper()== 'TEMPERATURE':
                da2= da2.rio.clip_box(minx, miny, maxx, maxy)
                da= (da + da2) / 2            
            # +++++++ Rename coordinates and set spatial dimensions to 'lon' and 'lat'
            da= da.rename(dict(band= 'month', x='lon', y= 'lat'))
            da.rio.set_spatial_dims(x_dim= 'lon', y_dim= 'lat', inplace= True)
            # +++++++ Stack the raster into an other DataArray with 3 additionnal dimensions (period, model, scenario)
#             da= da.expand_dims( {'period': 1, 
#                                  'scenario': 1,
#                                  'model': 1,
#                                 }, 
#                                axis= range(3, 6)
#                               )
#             da= da.assign_coords( {'period': ('period', [period]), 
#                                    'scenario': ('scenario', [scenario]),
#                                    'model': ('model', [model]),  
#                                   })
#             display(da)            
#             liste= list(da_tot['period'].values)
#             liste.extend([period for i in range(12)])
#             print(liste)
#             da_tot= xr.concat([da_tot, da], dim= pd.Index( liste, name= 'period'))
            if first_array== True:
                first_array= False
                da_tot= da
            else:
                da_tot= da
                #da_tot= xr.combine_nested([[da_tot], [da]], concat_dim= [[historical_period, '-', '-'], [period, model, scenario]])
#            period_coordinates.append(period)
            # +++++++ Annual aggregation
            if variable.upper()== 'PRECIPITATION':
                da_annual = da.sum('month', skipna= False)
            elif variable.upper()== 'TEMPERATURE':
                da_annual = da.mean('month', skipna= False)

            #########  Plot annual map (cartopy)
            if plot_anomaly==True:
                da_to_plot= da_annual-da_historical_annual
                title= variable + ' Anomaly '
            else:
                da_to_plot= da_annual
                title= variable + ' '
            title= 'WordlClim - Annual ' + title + '_'.join([period, scenario, model])
            fig= plot_annual(da_to_plot, title= title, levels=levels, cmap= cmap, vmin= vmin_annual, vmax = vmax_annual,
                            shapes= dict(polyg= dict(shape= gdf_polyg, edgecolor='magenta', facecolor= 'none', alpha= 1), 
                                         vector= dict(shape= gdf_wrapping, edgecolor='grey', facecolor='none', color='none', linewidth=0.5, alpha=1)),
                             folder_out= folder_out, file_out= title, clabel_fmt= clabel_fmt)
            plt.close()# close plot data, otherwise the map would be displayed after reactivation of interactive plot
plt.ion()# reactivate interactive plot
#da_tot= xr.concatenate([da_historical, da_tot], dim= period_coordinates)
#da_tot= da_tot.assign_coords(period= period_coordinates)
# display(da_tot)
print("Process completed in ", datetime.datetime.now()-start)
print('')
print('Historical data - Mean by month')
hv

###  <a name=section04></a>4) Elevation map
The DEM tif file can be downloaded from WorldClim https://www.worldclim.org/data/worldclim21.html

NB: The cell # 1 of this notebook must be ran before the cell below

Customize the code below before to run it:
- folder and name of the elevation raster file
- Min/Max elevations to anchor the color map

In [ ]:
"""
Created on March 2021
@author: Loïc Duffar
"""
import hvplot.xarray
import hvplot.pandas
import xarray as xr
import rioxarray
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import cartopy
import cartopy.feature as cfeature
import hvplot.xarray

######### Customization
# ----------- DEM global TIF file (FOLDER WITHOUT "\" CHARACTER AT THE END !!!)
fld_elv= r'X:\1-COMMUN\DIS\Documentation\Hydrologie\Documentation externe\Climat Monde\WorldClim\Elevation'
file_elv= 'wc2.1_30s_elev.tif'
# -------- Min/Max values to anchor the color map
#          - vmin should be negative to avoid low land to be colored in blue
#          - vmax should be adjusted according to the relief (lower than max elevation in case of high montains, otherwise greater than or equal to the max)
vmin_elv, vmax_elv= (-50, 2800)

######### Initializations
proj = cartopy.crs.PlateCarree()
title= 'Elevation map'
file_out= 'WorldClim - Elevation Map.png'
cmap_terrain= 'terrain'
res= '10m'

######### Process
# +++++++ Open file
da= rioxarray.open_rasterio(os.path.join(fld_elv, file_elv), masked= True)
da= da.sel(band= 1)

# +++++++ Clip data
da= da.rio.clip_box(minx, miny, maxx, maxy)

# +++++++ Rename x/y coordinates and set spatial dimensions to the new names
da= da.rename(dict(x= 'lon', y= 'lat'))
da.rio.set_spatial_dims(x_dim= 'lon', y_dim= 'lat', inplace= True)
da= da.reset_coords(names= ['band', 'spatial_ref'], drop=True)

# +++++++ Plot (cartopy)
plt.close()
#. . . . . . . . Plot elevation
da.plot.imshow(vmin= vmin_elv, vmax= vmax_elv,
#               cbar_kwargs= dict(shrink=0.6, label= 'Elv. (m)', pad=0.2),
               cmap= cmap_terrain, norm= mpl.colors.PowerNorm(0.3, clip=False), robust= True,
               cbar_kwargs= dict(shrink=0.6, label= 'Elv. (m)', aspect= 30),
               transform= proj, subplot_kws= dict(projection= proj),
              )
plt.gcf().set_size_inches(15, 12)
ax= plt.gca()

#. . . . . . . . Map custumization
plt.gcf().suptitle(title, y=1)
if gdf_polyg is not None:
    gdf_polyg.boundary.plot(ax=ax, edgecolor= 'magenta', facecolor= 'none', alpha= 1)
if gdf_wrapping is not None: 
    gdf_wrapping.plot(ax=ax, edgecolor= 'grey', facecolor= 'none', color= 'none', linewidth= 0.6, alpha= 1) 

#. . . . . . . . Map wrapping (NaturalEarth)
ax.add_feature(cfeature.NaturalEarthFeature('physical', 'rivers_lake_centerlines', res, facecolor='none', 
                edgecolor='blue'), alpha=0.5)
ax.add_feature(cfeature.NaturalEarthFeature('physical', 'rivers_europe', res, facecolor='none', edgecolor='blue'))
ax.add_feature(cfeature.NaturalEarthFeature('physical', 'lakes', res, facecolor=cfeature.COLORS['water'], edgecolor='blue'), 
                alpha=0.5)
ax.add_feature(cfeature.NaturalEarthFeature('physical', 'ocean', res, facecolor= 'aliceblue', edgecolor='black'), 
                alpha= 1)
    
plt.savefig(os.path.join(folder_out, file_out), dpi=150)

# +++++++ Plot (hvplot)
hv= da.hvplot(x='lon', y='lat', width= 950, height=700, geo= True,  crs= proj,
                coastline= res, 
                tiles=  'EsriTerrain', # CartoLight, StamenTerrain, StamenTerrainRetina, EsriTerrain, EsriNatGeo,  Wikipedia, OSM, StamenLabels, EsriReference
                cmap= cmap_terrain, clabel= 'm', alpha=0.6,
                title= title,
                clim=(-700, vmax_elv), 
         )
if gdf_polyg is not None :
    hv= hv * gdf_polyg.hvplot(x='lon', y='lat', width= 950, height=700, geo= True,  crs= proj,
                line_width= 2, line_color= 'magenta', fill_color= 'none',
                tiles=  'EsriReference', # CartoLight, StamenTerrain, StamenTerrainRetina, EsriTerrain, EsriNatGeo,  Wikipedia, OSM, StamenLabels, EsriReference
                clim=(-700, vmax_elv), 
         )
hv